In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col("datum").dt.year() >= 1918
).drop_nulls(
    subset=["datum"]
).sort(
    by="datum"
)

In [4]:
import re

In [5]:
def vroce(retezec):
    roky = []
    fraze = [
        r'[Rr]ok \d{2,4}',
        r'[Vv] roce \d{2,4}',
        r'[Rr]oku \d{2,4}',
        r'[Rr]okem \d{2,4}'
    ]
    for f in fraze:
        nalezy = re.findall(f,retezec)
        pridej = [int(re.search(r'\d{2,4}', x).group()) for x in nalezy]
        pridej = [int(f'19{x}') if 18 <= x < 100 else int(f'20{x}') if x < 17 else x for x in pridej]
        roky += pridej
    return roky

In [6]:
test = 'V roce 2011 bylo dobře, zato v roce 2012 bylo ještě líp, ovšem roku 2013 se to začalo kazit a rokem 2014 dobré časy skončily. Rok 39 byl těžký, rok 89 veselý. Rok 14 byl rokem začátku války.'

In [7]:
vroce(test)

[1939, 1989, 2014, 2011, 2012, 2013, 2014]

In [8]:
test2 = 'Žádná zmínka'

In [9]:
vroce(test2)

[]

In [10]:
df = df.with_columns(pl.col("text").map_elements(vroce, return_dtype=pl.List(pl.Int32)).alias("roky"))

In [11]:
vyvoj = df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    pl.col("roky") - pl.col("datum").dt.year()
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('roky').median()
)

In [12]:
alt.Chart(vyvoj.to_pandas(),width=800).mark_bar().encode(alt.X('datum:T'),alt.Y('roky:Q'))

alt.Chart(...)

In [13]:
vyvoj2 = df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    pl.col("roky") - pl.col("datum").dt.year()
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('roky').quantile(0.95)
)

In [14]:
alt.Chart(vyvoj2.to_pandas(),width=800).mark_bar().encode(alt.X('datum:T'),alt.Y('roky:Q'))

alt.Chart(...)

In [15]:
alt.Chart(
    df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    pl.col("roky") - pl.col("datum").dt.year()
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('roky').max()
).to_pandas(),
    width=800
).mark_bar().encode(alt.X('datum:T'),alt.Y('roky:Q'))

alt.Chart(...)